In [1]:
from src.user import User
from flask import session, request, json, jsonify 
import json
import os

In [2]:
from db.dbconnect import ClientDB

In [3]:
client_db = ClientDB(email='user1@email',password='1234')

In [4]:
client_db.getall_areo_to_cidade('UBERLANDIA')

[(1, 'AERO1', 'UBERLANDIA', 'MINAS GERAIS', 'MG')]

In [5]:
client_db.get_voo_to_minus_tarifas(2)

[(4, 'UBERLANDIA', 'SÃO PAULO', 'COMP2'),
 (1, 'UBERLANDIA', 'RIO JANEIRO', 'COMP1')]

In [6]:
client_db.email_get_pass()

('1234',)


[('1234',)]

In [7]:
client_db.getall_voo_to_companhia('COMP1')

[(1,
  1,
  'UBERLANDIA',
  'RIO JANEIRO',
  'COMP1',
  1970.0,
  datetime.date(2021, 10, 10),
  datetime.time(19, 30)),
 (2,
  1,
  'UBERLANDIA',
  'SÃO PAULO',
  'COMP1',
  1970.0,
  datetime.date(2021, 10, 10),
  datetime.time(19, 30))]

In [8]:
client_db.getall_areo_to_cidade('UBERLANDIA')

[(1, 'AERO1', 'UBERLANDIA', 'MINAS GERAIS', 'MG')]

In [9]:
client_db.get_preco_to_id_voo(1)

[1970.0, 42, 27]

In [10]:
client_db.update_voo_one(2,35)

In [11]:
client_db.email_get_user()

('USER1',)


[('USER1',)]

In [12]:
client_db.email_get_pass()

('1234',)


[('1234',)]

In [13]:
import datetime as date

In [14]:
date.date(2021,1,1).isoformat()

'2021-01-01'

In [15]:
client_db.get_voo_to_day(15,6,2021)

[(4, 'COMP2')]

In [16]:
def user_client_db(email,password):
    client = ClientDB(email=email,password=password)

    try:
        password_db = client.email_get_user()[0]
        if (password == password_db):
            return user
        else:
            return None
    except:
        return False

In [17]:
user_client_db('user1@email','1234')

('USER1',)


In [18]:
def companhia_to_aeropostos(companhia):
    cidades = []
    aero = []
    results = []
    client = ClientDB()

    try:
        db_results = client.getall_voo_to_companhia(companhia)
        for (_,_,src,dst,_,_,_,_) in db_results:
            cidades.append(src)
            cidades.append(dst)
        
        for city in cidades:
            db_pesquisa = client.getall_areo_to_cidade(city)
            for (_,nome,_,_,_) in db_pesquisa:
                aero.append(nome)
    
        for i,a in enumerate(aero):
            json_text = {'number': i, 'aeroposto': a}
            results.append(json_text)
    except:
        return None
    
    return results

In [19]:
companhia_to_aeropostos('COMP1')

[{'number': 0, 'aeroposto': 'AERO1'},
 {'number': 1, 'aeroposto': 'AERO2'},
 {'number': 2, 'aeroposto': 'AERO1'},
 {'number': 3, 'aeroposto': 'AERO3'}]

In [20]:
def origem_to_aeropostos(origem):
    results = []
    client = ClientDB()

    try:
        db_results = client.getall_areo_to_cidade(origem)
        for i, (_,nome,_,_,_) in enumerate(db_results):
            json_text = {'number': i, 'aeroposto': nome}
            results.append(json_text)
    except:
        return None

    return results

In [21]:
origem_to_aeropostos('UBERLANDIA')

[{'number': 0, 'aeroposto': 'AERO1'}]

In [22]:
def voo_to_data(dia,mes,ano):
    results = []
    client = ClientDB()
    try:
        dia = int(dia)
        mes = int(mes)
        ano = int(ano)
        db_results = client.get_voo_to_day(dia,mes,ano)
    
        for i, (id_voo,companhia) in enumerate(db_results):
            json_text = {'number': i+1, 'voo': id_voo, 'companhia': companhia}
            results.append(json_text)

        return results
    except:
        return None

In [23]:
voo_to_data(15,6,2021)

[{'number': 1, 'voo': 4, 'companhia': 'COMP2'}]

In [24]:
def voo_to_tax_limit(number):
    results = []
    client = ClientDB()

    try:
        number = int(number)
        db_results = client.get_voo_to_minus_tarifas(number)
        
        for (id_voo,src,dst,comp) in db_results:
            json_text = {'voo': id_voo, 'origem': src,
                         'destino': dst, 'companhia': comp}
            results.append(json_text)
        return results
    except:
        return None

In [25]:
voo_to_tax_limit(2)

[{'voo': 4,
  'origem': 'UBERLANDIA',
  'destino': 'SÃO PAULO',
  'companhia': 'COMP2'},
 {'voo': 1,
  'origem': 'UBERLANDIA',
  'destino': 'RIO JANEIRO',
  'companhia': 'COMP1'}]

In [26]:
def db_compra(id_voo):
    results = []
    client = ClientDB()
    db_results = client.get_preco_to_id_voo(id_voo)

    precos = db_results[0]
    vagas = db_results[1]
    disponiveis = db_results[2]

    if disponiveis < 1:
        return {'status': False, 'messagem': 'not_disponivel'}
    else:
        client.update_voo_one(id_voo,disponiveis-1)
        return {'status': True, 'ticket': vagas, 'preco': precos, 'acento': disponiveis}

In [27]:
db_compra(1)

{'status': True, 'ticket': 42, 'preco': 1970.0, 'acento': 27}